In [ ]:
teams_dict = {
    "Arizona Cardinals": ["NFC West", "NFC"],
    "Atlanta Falcons": ["NFC South", "NFC"],
    "Baltimore Ravens": ["AFC North", "AFC"],
    "Buffalo Bills": ["AFC East", "AFC"],
    "Carolina Panthers": ["NFC South", "NFC"],
    "Chicago Bears": ["NFC North", "NFC"],
    "Cincinnati Bengals": ["AFC North", "AFC"],
    "Cleveland Browns": ["AFC North", "AFC"],
    "Dallas Cowboys": ["NFC East", "NFC"],
    "Denver Broncos": ["AFC West", "AFC"],
    "Detroit Lions": ["NFC North", "NFC"],
    "Green Bay Packers": ["NFC North", "NFC"],
    "Houston Texans": ["AFC South", "AFC"],
    "Indianapolis Colts": ["AFC South", "AFC"],
    "Jacksonville Jaguars": ["AFC South", "AFC"],
    "Kansas City Chiefs": ["AFC West", "AFC"],
    "Las Vegas Raiders": ["AFC West", "AFC"],
    "Los Angeles Chargers": ["AFC West", "AFC"],
    "Los Angeles Rams": ["NFC West", "NFC"],
    "Miami Dolphins": ["AFC East", "AFC"],
    "Minnesota Vikings": ["NFC North", "NFC"],
    "New England Patriots": ["AFC East", "AFC"],
    "New Orleans Saints": ["NFC South", "NFC"],
    "New York Giants": ["NFC East", "NFC"],
    "New York Jets": ["AFC East", "AFC"],
    "Philadelphia Eagles": ["NFC East", "NFC"],
    "Pittsburgh Steelers": ["AFC North", "AFC"],
    "San Francisco 49ers": ["NFC West", "NFC"],
    "Seattle Seahawks": ["NFC West", "NFC"],
    "Tampa Bay Buccaneers": ["NFC South", "NFC"],
    "Tennessee Titans": ["AFC South", "AFC"],
    "Washington Football Team": ["NFC East", "NFC"]
}

In [ ]:
NFC = [team for team in teams_dict.keys() if "NFC" in teams_dict[team]]
AFC = [team for team in teams_dict.keys() if "AFC" in teams_dict[team]]

A_North = [team for team in teams_dict.keys() if "AFC North" in teams_dict[team]]
A_South = [team for team in teams_dict.keys() if "AFC South" in teams_dict[team]]
A_East = [team for team in teams_dict.keys() if "AFC East" in teams_dict[team]]
A_West = [team for team in teams_dict.keys() if "AFC West" in teams_dict[team]]

N_North = [team for team in teams_dict.keys() if "NFC North" in teams_dict[team]]
N_South = [team for team in teams_dict.keys() if "NFC South" in teams_dict[team]]
N_East = [team for team in teams_dict.keys() if "NFC East" in teams_dict[team]]
N_West = [team for team in teams_dict.keys() if "NFC West" in teams_dict[team]]

AFC_tiers = (A_North, A_South, A_East, A_West)
NFC_tiers = (N_North, N_South, N_East, N_West)

NFL_tiers = (AFC_tiers, NFC_tiers)

In [ ]:
# first try at the round robin matching algo
import random, copy, json

full_schedule = {}

# for week_num in range(16):
for week_num in range(2):
    afccopy = copy.copy(AFC)
    nfccopy = copy.copy(NFC)
    random.shuffle(afccopy)

    weekly_matchups = [{"home": afc_team.name, "away": nfc_team.name} for afc_team, nfc_team in zip(afccopy, nfccopy)]


    full_schedule[f"week_{week_num+1}"] = weekly_matchups

full_schedule_json = json.dumps(full_schedule, indent=4)  # indent for pretty printing
# print(full_schedule_json)


In [ ]:
# Generate a list of bye weeks, assign to teams
# Byes can happen between week 4 and week 14
import copy

# refresh the team objects
# create_new_league()

# print a list of teams with no bye week yet for debugging
no_bye_week = [team.name for team in nfl_teams if team.bye_week == None]
print(no_bye_week)

eligible_weeks = list(range(4, 15))

for team in nfl_teams:
    week_to_add = random.choice(eligible_weeks)
    team.set_bye_week(week_to_add)
    # print(team.bye_week, team.name)

# print a list of teams with no bye week for debugging. this should now an empty blank now
no_bye_week_after = [team.name for team in nfl_teams if team.bye_week == None]
print(no_bye_week_after)


['New England Patriots', 'Buffalo Bills', 'Miami Dolphins', 'New York Jets', 'Baltimore Ravens', 'Pittsburgh Steelers', 'Cleveland Browns', 'Cincinnati Bengals', 'Houston Texans', 'Indianapolis Colts', 'Tennessee Titans', 'Jacksonville Jaguars', 'Kansas City Chiefs', 'Denver Broncos', 'Las Vegas Raiders', 'Los Angeles Chargers', 'Dallas Cowboys', 'Washington Football Team', 'Philadelphia Eagles', 'New York Giants', 'Green Bay Packers', 'Chicago Bears', 'Minnesota Vikings', 'Detroit Lions', 'Tampa Bay Buccaneers', 'New Orleans Saints', 'Carolina Panthers', 'Atlanta Falcons', 'Seattle Seahawks', 'Los Angeles Rams', 'Arizona Cardinals', 'San Francisco 49ers']
[]


In [ ]:
# def generate_bye_weeks(list_of_teams):    
#     eligible_weeks = list(range(5, 15))
# #     in the NFLTeam objects, set the bye week
#     for team in list_of_teams:
#         week_to_add = random.choice(eligible_weeks)
#         team.set_bye_week(week_to_add)
#     return {week: [team.name for team in nfl_teams if team.bye_week == week] for week in range(4, 15)}

# def has_empty_bye_weeks(dictionary):
#     """ checks if there are any weeks where no team has a bye
#     Args:
#         dictionary (dictionary): a dictionary with key = week_num and value = a list of teams with a bye in that week (created)
#     Returns:
#         bool: returns True if generate_bye_weeks() creates a bye week schedule where any week between 4 and 14 has no teams with a bye.
#     """
#     for value in dictionary.values():
#         if isinstance(value, list) and not value:
#             return True
#     return False


In [ ]:
    def show_schedule(self):
        for matchup in self.schedule:
            if matchup[home_indicator] == True :
                print(self.name, "vs.", matchup.opponent)
            else:
                print(self.name, "@", matchup.opponent)